# Nye aviser

In [1]:
import dhlab.nbtext as nb
import dhlab.graph_networkx_louvain as gnl
import dhlab.nbtokenizer as tok
from dhlab.module_update import css, update, code_toggle

In [2]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import networkx as nx
import seaborn as sns
%matplotlib inline

In [24]:
def get_df(frases, title='aftenposten', media='aviser'):
    import requests
    querystring = " + ".join(['"'+frase+'"' for frase in frases])
    query = {
        'q':querystring,
        'size':1,
        'aggs':'year',
        'filter':['mediatype:{mt}'.format(mt=media),'title:{title}'.format(title=title)]
    }
    r = requests.get("https://api.nb.no/catalog/v1/items", params = query)
    aggs = r.json()['_embedded']['aggregations'][0]['buckets']
    return {x['key']:x['count'] for x in aggs}

def get_json(frases, mediatype='aviser', title='*'):
    import requests
    querystring = " + ".join(['"'+frase+'"' for frase in frases])
    query = {
        'q':querystring,
        'size':1,
        'snippets':mediatype,
        'aggs':'year',
        
       'filter':['mediatype:{mt}'.format(mt=mediatype),'title:{title}'.format(title=title)]
        'searchType':'FULL_TEXT_SEARCH'
        #'filter':
    }
    r = requests.get("https://api.nb.no/catalog/v1/items", params = query)
    aggs = r.json()
    return aggs

def get_data(frase, media='aviser', title='jazznytt'):
    import requests
    query = {
        'q':'"'+frase+'""',
        'size':1,
        'aggs':'year',
        'filter':['mediatype:{mt}'.format(mt=media),'title:{title}'.format(title=title)]
        
    }
    r = requests.get("https://api.nb.no/catalog/v1/items", params = query)
    return r.json()

def get_data_and(frases, title='aftenposten', media='aviser'):
    import requests
    querystring = " + ".join(['"'+frase+'"' for frase in frases])
    print(querystring)
    query = {
        'q':querystring,
        'size':1,
        'aggs':'year',
        'filter':['mediatype:{mt}'.format(mt=media), 'title:{title}'.format(title=title)],
        #'filter':'title:{title}'.format(title=title)
    }
    r = requests.get("https://api.nb.no/catalog/v1/items", params = query)
    return r.json()

def get_df_pd(frase, media='bøker'):
    import pandas as pd
    return pd.DataFrame.from_dict(get_df(frase, media=media ), orient='index').sort_index()

In [4]:
def phrase_plots(phrase_sets, title='aftenposten', fra = 1960, til = 2020, step=5, rot=0, colours = ['r', 'b','g','y','m','c']):
    df_all = []
    for f in phrase_sets:
        df_all.append(nb.frame(get_df(f, title= title), ', '.join(f)))
    df = pd.concat(df_all, sort=False)
    df.index = df.index.astype(int)
    df = df.sort_index()
    df['bins'] = pd.cut(df.index, range(fra, til, step), precision=0)
    df.groupby('bins').sum().plot(kind='bar', color=colours, figsize=(15,5), rot=rot)
    return

def phrase_plots_anno(phrase_sets, title='aftenposten', fra = 1960, til = 2020, rot=0, colours = ['r', 'b','g']):
    df_all = []
    for f in phrase_sets:
        df_all.append(nb.frame(get_df(f, title= title), ', '.join(f)))
    df = pd.concat(df_all, sort=False)
    df.index = df.index.astype(int)
    df = df.sort_index()
    #df['bins'] = pd.cut(df.index, range(fra, til, step), precision=0)
    df.plot(kind='bar', figsize=(15,5), rot=rot, color=colours)
    return

def graph_from_df(df, threshold = 100):
    edges =  []
    normalizer = {(x, y): df.stack()[(x,x)]*df.stack()[(y,y)] for (x,y) in df.stack().index}
    for (x, y) in df.stack().index:
        if x != y:
            if df.stack()[(x,y)] > threshold:
                edges.append([x,y,df.stack()[(x,y)]/normalizer[(x,y)]])
    G = nx.Graph()
    G.add_weighted_edges_from(edges)
    return G

In [25]:
get_df(['Helene Uri'])

{'2011': 82,
 '2006': 71,
 '2009': 66,
 '2007': 64,
 '2018': 61,
 '2010': 44,
 '2013': 42,
 '2019': 40,
 '2005': 37,
 '2008': 34,
 '2012': 33,
 '2004': 29,
 '2014': 26,
 '2003': 22,
 '2020': 19,
 '2015': 18,
 '2017': 16,
 '2001': 12,
 '2016': 12,
 '2002': 4,
 '1996': 2,
 '1991': 1,
 '1992': 1,
 '1993': 1,
 '1994': 1,
 '1995': 1,
 '1999': 1}

In [5]:
def get_konks(urn, phrase, window=1000, n = 1000):
    import requests
    querystring = '"'+ phrase +'"' 
    query = {
        'q':querystring,
        'fragments': n,
        'fragSize':window
       
    }
    r = requests.get("https://api.nb.no/catalog/v1/items/{urn}/contentfragments".format(urn=urn), params = query)
    res = r.json()
    results = []
    try:
        for x in res['contentFragments']:
            urn = x['pageid']
            hit = x['text']
            splits = hit.split('<em>')
            s2 = splits[1].split('</em>')
            before = splits[0]
            word = s2[0]
            after = s2[1]
            results.append({'urn': urn, 'before': before, 'word':word, 'after':after})
    except:
        True
    return results

In [6]:
r = get_konks("URN:NBN:no-nb_digibok_2011021403034", "pølse i handa", window=2000)
r

[{'urn': 'URN:NBN:no-nb_digibok_2011021403034_0009',
  'before': 'gjør vi jo alle pølser: En enslig mann på en bensinstasjon tre! ved middagstider, med ei ',
  'word': 'pølse i handa,',
  'after': ' bryter god spiseskikk. Pølsa blir da for mange et symbol på forfall. Vi har ikke som'}]

In [9]:
r = get_konks("/URN:NBN:no-nb_digavis_dagbladet_null_null_20200324_152_71_1", "corona", window=500)
r

[{'urn': 'URN:NBN:no-nb_digavis_dagbladet_null_null_20200324_152_71_1-1_001_hovedavis',
  'before': 'STORE ELBIL-GUIDEN STOR EFFEKT MOT KOLS: SKJØNTE JEG AT DET VAR ',
  'word': 'CORONA',
  'after': ' Tirsdag 24. mars 2020 SUPERMETODEFOR RØYKE-SLUTT SLIK '},
 {'urn': 'URN:NBN:no-nb_digavis_dagbladet_null_null_20200324_152_71_1-1_001_hovedavis',
  'before': 'Tirsdag 24. mars 2020 Nr. 71. Uke 13. 152. årgang. Løssalg kr 40,00 SLIK SKJØNTE DET ',
  'word': 'CORONA',
  'after': ' '},
 {'urn': 'URN:NBN:no-nb_digavis_dagbladet_null_null_20200324_152_71_1-1_004_hovedavis',
  'before': 'grave litt dypere. KOMMENTAREN Inger Merete Hobbelstad LEDER 3 imh@dagbladet.no Han KOMMENTAREN NYHET 4 ',
  'word': 'CORONA-KRISA',
  'after': ' Jeg bygger flokkimmunitet Flaggermuseffekten fra Wuhan har nådd fram til Dagbladets journalist. Les Trude'},
 {'urn': 'URN:NBN:no-nb_digavis_dagbladet_null_null_20200324_152_71_1-1_004_hovedavis',
  'before': 'egne subtile symptomer og gremmes over hva jeg kan ha utsa

In [66]:
nb.concordance(word='morsomt', corpus='avis', title='aftenposten', kind="json")

[{'after': '. Når Kristopher Schaus pølsestunt',
  'before': '; de synes medier ev',
  'urn': 'https://www.nb.no/items/URN:NBN:no-nb_digavis_aftenposten_null_null_20040204_145_58_2',
  'word': 'morsomt'},
 {'after': ". Kristoffer Nordvik Ti '",
  'before': 'det etter all sannsynlighet blir',
  'urn': 'https://www.nb.no/items/URN:NBN:no-nb_digavis_aftenposten_null_null_20040204_145_58_2',
  'word': 'morsomt'},
 {'after': 'kurs vil dere få kunnskap',
  'before': 'Gjennom gratis , lærerikt og',
  'urn': 'https://www.nb.no/items/URN:NBN:no-nb_digavis_aftenposten_null_null_20040204_145_58_2',
  'word': 'morsomt'},
 {'after': 'kurs vil dere få kunnskap',
  'before': 'Gjennom gratis , lærerikt og',
  'urn': 'https://www.nb.no/items/URN:NBN:no-nb_digavis_aftenposten_null_null_20040204_145_58_2',
  'word': 'morsomt'},
 {'after': ', innrømmer 18 - åringen',
  'before': ', men det er ikke',
  'urn': 'https://www.nb.no/items/URN:NBN:no-nb_digavis_aftenposten_null_null_20040204_145_58_2',
  'word':